##### Copyright 2025 Google LLC

In [1]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Workshop: Build with Gemini (Part 1)

<a target="_blank" href="https://colab.research.google.com/github/markmcd/gemini-workshop/blob/main/01-text-prompting.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This workshop teaches how to build with Gemini using the Gemini API and Python SDK.

Course outline:

- **Part 1 (this notebook): Quickstart + Text prompting**
  - Text generation
  - Token counting
  - Streaming response
  - Chats
  - System prompts
  - Configuration parameters
  - Long context
  - Final excercise: Chat with book

- **[Part 2: Multimodal capabilities (image, video, audio, docs, code)](./02-multimodal-capabilities.ipynb)**

- **[Part 3: Thinking models + agentic capabilities (tool usage)](./03-thinking-and-tools.ipynb)**

## 0. Use the Google AI Studio as playground

Explore and play with all models in the [Google AI Studio](https://aistudio.google.com/apikey).


## 1. Setup


Install the [Google Gen AI Python SDK](https://github.com/googleapis/python-genai)

In [4]:
%pip install -q -U google-genai

Get a free API key in the [Google AI Studio](https://aistudio.google.com/apikey).

Configure the API key, the client, and define a model.

In [6]:
from google import genai
from google.genai import types
import os
import sys

try:
    from google.colab import userdata
    GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')
except ImportError:
    GEMINI_API_KEY = os.getenv('GEMINI_API_KEY')


client = genai.Client(api_key=GEMINI_API_KEY)

# MODEL = "gemini-2.0-flash"
# MODEL = "gemini-2.5-pro"
# MODEL = "gemini-2.5-flash-lite"
MODEL = "gemini-2.5-flash"

 See all [models](https://ai.google.dev/gemini-api/docs/models).

## 2. Send your first prompt

In [7]:
response = client.models.generate_content(
    model=MODEL,
    contents="Create 3 names for a vegan restaurant"
)

print(response.text)

Here are 3 names for a vegan restaurant, each with a slightly different vibe:

1.  **Root & Bloom:** (Evocative, natural, emphasizes plant-based origins and fresh, vibrant food)
2.  **The Kind Plate:** (Ethical, welcoming, highlights compassion and delicious, animal-free meals)
3.  **Flora & Feast:** (Elegant, botanical, suggests an abundant and delightful dining experience centered around plants)


## 3. Token counting

Count tokens before generation.

Note that the latest pricing can be obtained from https://ai.google.dev/gemini-api/docs/pricing. The numbers below are approximations and may be out of date.

In [8]:
prompt = "The quick brown fox jumps over the lazy dog."

print(f"# characters {len(prompt)}")
print(f"# words {len(prompt.split())}")
# Try a rule of thumb (4 chars / token). These are always estimates.
print(f"# tokens: ~{int(len(prompt) / 4)}")

# Count tokens in the input
token_count = client.models.count_tokens(
    model=MODEL,
    contents=prompt
)
print(f"Input tokens: {token_count.total_tokens}")

# Estimate cost (example pricing for 2.5 Flash - always check current rates)
COST_PER_MILLION_INPUT_TOKENS_USD = 0.30
estimated_cost = token_count.total_tokens * COST_PER_MILLION_INPUT_TOKENS_USD / 1_000_000
print(f"Estimated input cost: ${estimated_cost:.6f}")

# characters 44
# words 9
# tokens: ~11
Input tokens: 11
Estimated input cost: $0.000003


Count tokens after generation:

In [9]:
prompt = "Write a haiku about artificial intelligence."

response = client.models.generate_content(
    model=MODEL,
    contents=prompt
)

print(response.text)
print()

# Access token usage metadata
usage = response.usage_metadata
print(f"Input tokens: {usage.prompt_token_count}")
print(f"Thought tokens: {usage.thoughts_token_count}")
print(f"Output tokens: {usage.candidates_token_count}")
print(f"Total tokens: {usage.total_token_count}")

# Calculate total estimated cost
COST_PER_MILLION_OUTPUT_TOKENS_USD = 2.50
thought_tokens = int(usage.thoughts_token_count)
total_cost = (usage.prompt_token_count * COST_PER_MILLION_INPUT_TOKENS_USD + (usage.candidates_token_count + thought_tokens) * COST_PER_MILLION_OUTPUT_TOKENS_USD) / 1_000_000
print(f"Total estimated cost: ${total_cost:.6f}")

Cold circuits awake,
Processing the world's vast data,
Future's mind takes form.

Input tokens: 9
Thought tokens: 633
Output tokens: 21
Total tokens: 663
Total estimated cost: $0.001638


## 4. Text generation

The simplest way to generate text is to provide the model with a text-only prompt. `contents` can be a single prompt, a list of prompt parts, or a combination of multimodal inputs.

In [10]:
response = client.models.generate_content(
    model=MODEL,
    #contents="Create 3 names for a vegan restaurant",
    #contents=["Create 3 names for a vegan restaurant"],
    contents=["Create 3 names for a vegan restaurant", "city: Perth"]
)

print(response.text)

Here are 3 names for a vegan restaurant in Perth, with a little explanation for each:

1.  **Verdant Kitchen**
    *   **Why it works:** "Verdant" means green and lush, immediately evoking a fresh, plant-rich environment. "Kitchen" suggests a place of culinary creation and wholesome food. It sounds modern, clean, and inviting, without explicitly saying "vegan" but strongly implying it.

2.  **The Kind Fork**
    *   **Why it works:** "Kind" subtly refers to the ethical and compassionate aspect of veganism (kind to animals, kind to the planet). "Fork" clearly indicates it's a place to eat. It's memorable, has a warm and friendly feel, and is approachable for both vegans and those exploring plant-based options.

3.  **The Perth Harvest**
    *   **Why it works:** "Perth" anchors the restaurant to its location, making it appealing to locals. "Harvest" implies fresh, seasonal, abundant, and locally sourced produce – all strong pillars of a good vegan restaurant. It sounds wholesome, natura

#### Streaming response

By default, the model returns a response after completing the entire text generation process. You can achieve faster interactions by using streaming to return the output as it is generated.

In [11]:
response = client.models.generate_content_stream(
    model=MODEL,
    contents=["Explain how AI works"]
)

for chunk in response:
    print(chunk.text, end="")

AI, or Artificial Intelligence, isn't a single technology but a broad field focused on enabling machines to perform tasks that typically require human intelligence. This includes learning, problem-solving, understanding language, recognizing patterns, and making decisions.

At its core, most modern AI works by **identifying patterns in data and then using those patterns to make predictions or take actions.**

Let's break down the fundamental principles:

---

### The Core Ingredients of AI

1.  **Data:** This is the fuel for AI. AI systems learn from vast amounts of information (text, images, numbers, audio, video). The quality, quantity, and relevance of this data are crucial.
2.  **Algorithms:** These are the step-by-step instructions or "recipes" that the AI uses to process data, learn patterns, and make decisions. They are the mathematical models that define how the AI operates.
3.  **Computational Power:** Modern AI, especially deep learning, requires immense processing power, oft

#### Chat

The SDK [`Chat` class](https://googleapis.github.io/python-genai/genai.html#genai.chats.Chat) provides an interface to keep track of conversation history. Behind the scenes it uses the same [`generate_content`](https://googleapis.github.io/python-genai/genai.html#genai.models.Models.generate_content) method.

In [12]:
chat = client.chats.create(model=MODEL)

response = chat.send_message("I have 2 dogs in my house.")
print(response.text)

Oh, how lovely! Two dogs must bring a lot of joy and companionship to your home.

Do you want to tell me anything about them, like their names or breeds? Or perhaps you have a question about having two dogs?


In [13]:
response = chat.send_message("I have 2 poodles")
print(response.text)

Ah, two poodles! That's wonderful. Poodles are such intelligent, elegant, and often very playful dogs.

Do you have standard, miniature, or toy poodles? And what are their names, if you'd like to share?


## 5. Configuration parameters

Every prompt you send to the model includes parameters that control how the model generates responses. You can configure these parameters, or let the model use the default options.

In [14]:
response = client.models.generate_content(
    model=MODEL,
    contents=["Explain how AI works"],
    config=types.GenerateContentConfig(
        max_output_tokens=1024,
        temperature=1.0,
        top_p=0.95,
        stop_sequences=None,
        thinking_config=types.ThinkingConfig(
          include_thoughts=True,
          thinking_budget=100,
        ),
    )
)
print(response.text)

AI, or Artificial Intelligence, isn't one single technology but a broad field encompassing various techniques that enable machines to simulate human-like intelligence. At its core, AI works by using **algorithms** to process **data**, identify **patterns**, make **decisions**, and learn to improve its performance over time.

Let's break down the fundamental steps and concepts:

---

### The Core Loop: Data -> Algorithms -> Learning -> Prediction/Action

1.  **Data Ingestion:**
    *   **What it is:** AI systems need vast amounts of data to learn from. This data can be text, images, audio, video, sensor readings, numerical tables, etc.
    *   **Why it's crucial:** Just like humans learn from experience, AI learns from observing patterns and relationships within the data. More diverse and relevant data generally leads to better performance.
    *   **Example:** To teach an AI to identify cats, you'd feed it millions of images, some with cats, some without, with varying breeds, lighting,

- `max_output_tokens`: Provides a mechanism for a maximum output length (including thought tokens). Can be helpful for avoiding costs in error scenarios when the expected answer is short.
- `temperature`: [0, 2]. Controls randomness in token selection. Use <0.4 for more reproducibility, >0.7 for more diversity when re-run.
- `top_p`: [0, 1]. Controls diversity. Lower values = more focused, higher = more diverse
- `stop_sequences`: List of strings (up to 5) that tells the model to stop generating text if one of the strings is encountered in the response.
- `thinking_config.include_thoughts`: Specify whether or not model thoughts should be generated as part of the response. Note that not all models support enabling or disabling thinking.
- `thinking_config.thinking_budget`: How many tokens to budget for thoughts.

#### System instructions

System instructions let you steer the behavior of a model based on your specific use case. When you provide system instructions, you give the model additional context to help it understand the task and generate more customized responses. The model should adhere to the system instructions over the full interaction with the user, enabling you to specify product-level behavior separate from the prompts provided by end users.

In [15]:
response = client.models.generate_content(
    model=MODEL,
    config=types.GenerateContentConfig(system_instruction="You are Dumbledore. Be sure to welcome any new students."),
    contents="Hello there",
)

print(response.text)

Ah, hello there! A most hearty welcome to you. I trust you've found your way through the hustle and bustle of the Great Hall without too much trouble?

It is always a delight to see new faces joining us here at Hogwarts. I do hope you are prepared for an extraordinary journey of learning and discovery. The adventure, I assure you, is only just beginning! Please, find a comfortable seat, for the feast will be commencing shortly.


## 6. Long context

Gemini 2.0 and 2.5 models have a 1M token context window.

In practice, 1 million tokens could look like:

- 50,000 lines of code (with the standard 80 characters per line)
- All the text messages you have sent in the last 5 years
- 8 average length English novels
- 1 hour of video data
- ... or some combination of the above.

For this step, you will feed in an entire book and ask questions.


In [16]:
import requests
res = requests.get("https://gutenberg.org/cache/epub/16317/pg16317.txt")
book = res.text

In [17]:
print(book[:200])

﻿The Project Gutenberg eBook of The Art of Public Speaking
    
This ebook is for the use of anyone anywhere in the United States and
most other parts of the world at no cost and with almost no res


In [18]:
print(f"# characters {len(book)}")
print(f"# words {len(book.split())}")
print(f"# tokens: ~{int(len(book) / 4)}")

# characters 979772
# words 162461
# tokens: ~244943


Since this is a longer prompt than before, calculate the token length accurately.

In [19]:
prompt = f"""Summarize the book.

Book:
{book}
"""

token_response = client.models.count_tokens(
    model=MODEL,
    contents=prompt
)
print(token_response.total_tokens)

250498


Now execute the prompt requesting a book summary.

In [20]:
response = client.models.generate_content(
    model=MODEL,
    contents=prompt
)

print(response.text)

"The Art of Public Speaking" by J. Berg Esenwein and Dale Carnegie asserts that effective public speaking is fundamentally an **outward expression of the speaker's inner self**, emphasizing **self-development, a full mind, a warm heart, and a dominant will** as primary elements, rather than mere adherence to external rules or imitation.

The book provides comprehensive guidance across three main areas:

1.  **Developing the Speaker's Delivery:** It offers practical advice on overcoming self-consciousness and stage fright through frequent practice, absorption in the subject, and projecting confidence. It details how to achieve vocal efficiency and avoid monotony by varying pitch, pace, emphasis, and inflection. Proper breathing, voice placement (ease, openness, forwardness), and the cultivation of voice charm are discussed. The importance of distinct articulation, accentuation, and enunciation is stressed. Finally, it covers the "truth about gesture," advocating for natural, spontaneous

## !! Exercise: Chat with a book !!

Create an interactive chat session where you can "talk" to the book "Alice in Wonderland". You'll set up the chat with a specific persona for the AI and use the book's text as context for the conversation.

Tasks:
- Download the text of "Alice in Wonderland" (helper code block is provided).
- Create a chat session using `client.chats.create()`.
- Use a system prompt: `"You are an expert book reviewer with a witty tone."`
- Use a temperature of `1.2`
- Send an initial message to the chat session using `chat.send_message()`.
- Send at least one follow-up question to the chat session and print its response.

In [21]:
res = requests.get("https://gutenberg.org/cache/epub/28885/pg28885.txt")
book = res.text
print(f"# tokens: ~{int(len(book) / 4)}")

# tokens: ~44365


In [ ]:
# TODO(you!): Create a chat and ask questions about the book

## Recap & Next steps

Nice work! You learned:
- The `google.genai` Python SDK
- Text prompting
- Token counting
- Streaming and chats
- System prompts and config options
- Long context

Key Takeaways:
- Monitor token usage to control costs and stay within limits
- Use streaming for interactive applications and long responses
- Configure parameters based on your use case (factual vs creative content)
- System instructions are powerful for setting behavior and tone

More helpful resources:
- [Text Generation Guide](https://ai.google.dev/gemini-api/docs/text-generation)
- [Token Counting Guide](https://ai.google.dev/gemini-api/docs/tokens)
- [Long Context Documentation](https://ai.google.dev/gemini-api/docs/long-context)

Next steps:
- [Part 2: Multimodal capabilities (image, video, audio, docs, code)](./02-multimodal-capabilities.ipynb)